In [2]:
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import numpy as np


/Users/astro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
p = "TRAPPIST-1 b"
result = NasaExoplanetArchive.query_criteria(
        table="pscomppars", 
        select="top 10 pl_name, ra, dec, sy_dist",
        where=f"pl_name like '{p}'"
    )
result

pl_name,ra,dec,sy_dist,sky_coord
,deg,deg,pc,"deg,deg"
str12,float64,float64,float64,SkyCoord
TRAPPIST-1 b,346.6263919,-5.0434618,12.4298888,"346.6263919,-5.0434618"


In [133]:
ra = result['ra'][2]    # Right ascension
dec = result['dec'][2]  # Declination
distance = result['sy_dist'][2]  # Distance in parsecs
ra, dec, distance

(<MaskedQuantity 288.067445 deg>,
 <MaskedQuantity 42.355305 deg>,
 <MaskedQuantity 367. pc>)

x = d cos dec cos ra
y = d cos dec sin ra
z =  d sin dec

In [158]:
exoplanet_coord = SkyCoord(ra=ra, dec=dec, distance=distance)
exoplanet_cartesian = exoplanet_coord.cartesian
exo_x = exoplanet_cartesian.x.value.item()
exo_y = exoplanet_cartesian.y.value.item()
exo_z = exoplanet_cartesian.z.value.item()
exoplanet_cartesian

<CartesianRepresentation (x, y, z) in pc
    (84.11084879, -257.83347055, 247.25749044)>

In [162]:
import pandas as pd
df = pd.read_csv('hipparcos.csv')


array(['F5', 'K3V', 'B9', ..., 'B6IVe', 'A:w...', 'A4:IV/Vn...'],
      dtype=object)

In [160]:
df = df[["HIP", "Vmag", "RAdeg", "DEdeg", "Plx"]]
df

,HIP,Vmag,RAdeg,DEdeg,Plx
0,1,9.10,0.000912,1.089013,3.54
1,2,9.27,0.003797,-19.498837,21.90
2,3,6.61,0.005008,38.859286,2.81
3,4,8.06,0.008382,-51.893546,7.75
4,5,8.55,0.009965,-40.591224,2.87
...,...,...,...,...,...
118213,118318,6.99,359.963744,11.673709,1.92
118214,118319,8.23,359.973913,-22.428180,10.63
118215,118320,7.59,359.976057,5.956638,5.00
118216,118321,9.20,359.978239,-64.372572,19.22


In [161]:
df.dropna(inplace=True)
df.rename(columns={'RAdeg': 'ra', 'DEdeg': 'dec', 'Plx':'parallax'}, inplace=True)

df = df[df["parallax"] > 0]
df["dist"] = 1000/df["parallax"]

df["abs_mag"] = df["Vmag"] - 5 * np.log10(df["dist"]) + 5
df["x"] = df["dist"]*np.cos(np.radians(df["dec"]))*np.cos(np.radians(df["ra"]))
df["y"] = df["dist"]*np.cos(np.radians(df["dec"]))*np.sin(np.radians(df["ra"]))
df["z"] = df["dist"]*np.sin(np.radians(df["dec"]))

df["x"] = df["x"] - exo_x
df["y"] = df["y"] - exo_y
df["z"] = df["z"] - exo_z

df["dist_exo"] = np.linalg.norm(df[['x', 'y', 'z']], axis=1)

df["app_mag"] = df["abs_mag"] + 5 * np.log10(df["dist_exo"]) - 5


filtered_df = df[df["app_mag"]<3]
filtered_df = filtered_df[["x", "y", "z", "app_mag"]]
skycoord = SkyCoord(x=filtered_df["x"], y=filtered_df["y"], z=filtered_df["z"], representation_type='cartesian').spherical

filtered_df["SKY"] = skycoord
filtered_df["ra"] = skycoord.lon.deg
filtered_df["dec"] = skycoord.lat.deg
filtered_df["dist"] = skycoord.distance
filtered_df

/var/folders/xj/fb43r5v57595jl_b3dsxp35w0000gn/T/ipykernel_21995/3810882256.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["dist"] = 1000/df["parallax"]
/var/folders/xj/fb43r5v57595jl_b3dsxp35w0000gn/T/ipykernel_21995/3810882256.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["abs_mag"] = df["Vmag"] - 5 * np.log10(df["dist"]) + 5
/var/folders/xj/fb43r5v57595jl_b3dsxp35w0000gn/T/ipykernel_21995/3810882256.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,x,y,z,app_mag,SKY,ra,dec,dist
24418,-37.889974,487.777216,-281.062526,2.063853,"(94.44175085, -29.87651682, 564.23258204) (deg...",94.441751,-29.876517,564.232582
26292,-41.485758,667.051588,-255.889538,2.891542,"(93.55879713, -20.95049717, 715.65239071) (deg...",93.558797,-20.950497,715.652391
30416,-90.172376,315.622660,-323.520979,2.789382,"(105.94446828, -44.58420181, 460.88452447) (de...",105.944468,-44.584202,460.884524
34417,-228.813527,728.259438,-491.504189,2.920562,"(107.44231856, -32.77627156, 907.90627711) (de...",107.442319,-32.776272,907.906277
68639,-152.377770,216.885855,-387.235249,2.932549,"(125.09077526, -55.60829992, 469.26495403) (de...",125.090775,-55.608300,469.264954
...,...,...,...,...,...,...,...,...
103537,80.427353,93.045005,7.061219,2.504868,"(49.16012857, 3.2859784, 123.19006799) (deg, d...",49.160129,3.285978,123.190068
103965,103.653229,77.979994,3.194496,1.497937,"(36.95473332, 1.41078668, 129.75005244) (deg, ...",36.954733,1.410787,129.750052
104099,63.444657,118.542670,-24.659672,2.844463,"(61.84407747, -10.39296786, 136.69560519) (deg...",61.844077,-10.392968,136.695605
107219,84.384159,144.380655,-211.896793,2.965032,"(59.69558754, -51.71903061, 269.93834588) (deg...",59.695588,-51.719031,269.938346


In [116]:
df.head()

,ra,dec,parallax,phot_g_mean_mag,dist,abs_mag,x,y,z,dist_exo,app_mag
0,245.037074,-59.064074,0.570414,18.323439,1753.112914,7.104390,-352.609292,-828.739646,-1570.878319,1810.746091,18.393677
1,245.039067,-59.065358,0.547712,19.834799,1825.777989,8.527559,-368.330138,-862.587505,-1633.227282,1883.389404,19.902260
2,245.037612,-59.067410,0.043391,19.809600,23046.059813,2.996617,-4971.692188,-10751.415891,-19835.438459,23103.144499,19.814972
3,245.040272,-59.064684,0.702857,20.239350,1422.764603,9.473685,-280.895185,-674.784800,-1287.532238,1480.532302,20.325774
4,245.032803,-59.066588,0.180173,18.335570,5550.220795,4.614019,-1176.525865,-2598.119542,-4827.945182,5607.449772,18.357846


In [100]:
df[df["app_mag"]<5]

,ra,dec,parallax,phot_g_mean_mag,dist,abs_mag,x,y,z,dist_exo,app_mag
877871,276.743291,-45.968696,11.669601,3.488739,85.692736,-1.175982,34.735799,-70.842400,-128.768114,151.018040,4.719163
2517077,315.874058,23.996209,2.943950,4.915763,339.679710,-2.739585,250.494683,-227.751509,70.981201,345.914248,4.955258


In [43]:
for i in range(len(star_coords_list)):
        star_coords_list[i] = star_coords_list[i] - exoplanet_cartesian
        relative_skycoord = SkyCoord(x=star_coords_list[i].x, 
                                y=star_coords_list[i].y, 
                                z=star_coords_list[i].z, 
                                representation_type='cartesian')
        relative_skycoord = relative_skycoord.spherical
        ra_value = relative_skycoord.lon.deg.filled(np.nan).item()  # RA in degrees
        dec_value = relative_skycoord.lat.deg.filled(np.nan).item()  # Dec in degrees
        distance_value = relative_skycoord.distance.pc.filled(np.nan).item()  # Distance in parsecs

        star_coords_list[i] = {
            "RA": ra_value,
            "Dec": dec_value,
            "Distance (pc)": distance_value
        }
